In [ ]:
import sys
print ('Running in colab:', 'google.colab' in sys.modules)
if 'google.colab' in sys.modules:
  folderloc = "drive/MyDrive/CMPE 255/"
else:
  folderloc = "Documents/SJSU/FALL_2022/"

Running in colab: False


#Import File

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier

np.set_printoptions(linewidth=115)


In [ ]:
# input_file = folderloc + "A_Z Handwritten Data.csv"
# df = pd.read_csv(input_file, header = 0)

In [ ]:
#code split
# train, test = model_selection.train_test_split(df, test_size=0.20)
# train.to_csv(folderloc+'train.csv',index = False)
# test.to_csv(folderloc+'test.csv', index = False)

#Import Group Train/Test and view sample data

In [ ]:
input_train = pd.read_csv(folderloc + "train.csv",header = 0)


In [ ]:
input_test = pd.read_csv(folderloc + "test.csv",header = 0)

In [ ]:
row = 0
test_mat_1 = input_test.iloc[row,1:].values
test_mat_1_label = input_test.iloc[row,0]
print(test_mat_1)
print(test_mat_1_label)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 147 244 160   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 186 255 209   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 186 255 209   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 186 255 209   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 186 255 216  12   0

In [ ]:
test_mat_1 = np.resize(test_mat_1, (28, 28))
im = Image.fromarray(test_mat_1.astype(np.uint8))
im

In [ ]:
input_test.info()
input_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74490 entries, 0 to 74489
Columns: 785 entries, 0 to 0.648
dtypes: int64(785)
memory usage: 446.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297960 entries, 0 to 297959
Columns: 785 entries, 0 to 0.648
dtypes: int64(785)
memory usage: 1.7 GB


In [ ]:
test_mat = input_test.iloc[:,1:].values
test_mat_class = input_test.iloc[:,0].values

In [ ]:
train_mat = input_train.iloc[:,1:].values

In [ ]:
train_mat_class = input_train.iloc[:,0].values

In [ ]:
train_mat[0,:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,  83, 255, 168,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0, 163, 255, 206,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  39, 242, 255, 119,   0,   

#K Nearest Neighbor Classifier

In [ ]:
neigh = KNeighborsClassifier()

In [ ]:
neigh.fit(train_mat, train_mat_class)

KNeighborsClassifier()

In [ ]:
pred = neigh.predict(test_mat)
print(pred)

[ 9 14  1 ... 13 14  7]


#Scoring KNN

In [ ]:
print(neigh.score(test_mat,test_mat_class))

0.9588401127668144


In [ ]:
input_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297960 entries, 0 to 297959
Columns: 785 entries, 0 to 0.648
dtypes: int64(785)
memory usage: 1.7 GB


In [ ]:
df.info()

NameError: ignored

In [ ]:
df.iloc[:,0].value_counts()

14    57825
18    48419
20    29008
2     23409
19    22495
15    19341
13    19010
0     13869
12    12336
11    11586
17    11566
4     11440
24    10859
22    10784
3     10134
1      8668
9      8493
7      7218
23     6272
25     6076
16     5812
6      5762
10     5603
21     4182
5      1163
8      1120
Name: 0, dtype: int64

In [ ]:
row1 = df.iloc[16290,:].values
row1 = row1[1:785]

print(row1)
print(type(row1))
print(row1.shape)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  20 114 224   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  10  87 188 226 255   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  44 167 177  51 223 137   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   5 113 206 118   0  83 255  20   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  25 142 206  44   0   0 191 102   1

In [ ]:
mat1 = np.resize(row1, (28, 28))

In [ ]:

print(mat1)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  20 114 224   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  10  87 188 226 255   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  44 167 177  51 223 137   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   5 113 206 118   0  83 255  20   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  25 142 206  44   

In [ ]:
im = Image.fromarray(mat1.astype(np.uint8))

In [ ]:
im